In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier,DecisionTreeClassifier,GBTClassifier
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.sql.types import IntegerType,LongType,StructField,StructType
from pyspark.ml import pipeline,Pipeline
import pyspark.sql.functions as SF

In [0]:
SS = SparkSession.builder.appName('Tree').getOrCreate()

In [0]:
df = SS.read.format('libsvm').load('/FileStore/tables/sample_libsvm_data-2.txt')

In [0]:
df.show(5)

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows

In [0]:
df.printSchema()

root
-- label: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
train,test = df.randomSplit([0.7,0.3])

In [0]:
DCT = DecisionTreeClassifier()
RFC = RandomForestClassifier(numTrees=100)
GRA = GBTClassifier()

In [0]:
dct = DCT.fit(train)
rfc = RFC.fit(train)
gra = GRA.fit(train)

In [0]:
dct_pred = dct.transform(test)
rfc_pred = rfc.transform(test)
gra_pred = gra.transform(test)

In [0]:
dct_df = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
rfc_df = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
gra_df = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')

In [0]:
dct_df1 = dct_df.evaluate(dct_pred)
rfc_df1 = rfc_df.evaluate(rfc_pred)
gra_df1 = gra_df.evaluate(gra_pred)

In [0]:
dct_df1

Out[33]: 1.0

In [0]:
rfc_df1

Out[34]: 1.0

In [0]:
gra_df1

Out[35]: 1.0

In [0]:
dct_dfm = MulticlassClassificationEvaluator(metricName='accuracy')
rfc_dfm = MulticlassClassificationEvaluator(metricName='accuracy')
gra_dfm = MulticlassClassificationEvaluator(metricName='accuracy')

In [0]:
dct_df2 = dct_dfm.evaluate(dct_pred)
rfc_df2 = rfc_dfm.evaluate(rfc_pred)
gra_df2 = gra_dfm.evaluate(gra_pred)

In [0]:
ss1 = SparkSession.builder.appName('project').getOrCreate()

In [0]:
df1 = ss1.read.table('college_csv')
df2 = sqlContext.sql('select * from college_csv')

In [0]:
df1.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: string (nullable = true)
-- Accept: string (nullable = true)
-- Enroll: string (nullable = true)
-- Top10perc: string (nullable = true)
-- Top25perc: string (nullable = true)
-- F_Undergrad: string (nullable = true)
-- P_Undergrad: string (nullable = true)
-- Outstate: string (nullable = true)
-- Room_Board: string (nullable = true)
-- Books: string (nullable = true)
-- Personal: string (nullable = true)
-- PhD: string (nullable = true)
-- Terminal: string (nullable = true)
-- S_F_Ratio: string (nullable = true)
-- perc_alumni: string (nullable = true)
-- Expend: string (nullable = true)
-- Grad_Rate: string (nullable = true)

In [0]:
df1.show(2)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
 School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
Abilene Christian...| Yes|1660| 1232| 721| 23| 52| 2885| 537| 7440| 3300| 450| 2200| 70| 78| 18.1| 12| 7041| 60|
 Adelphi University| Yes|2186| 1924| 512| 16| 29| 2683| 1227| 12280| 6450| 750| 1500| 29| 30| 12.2| 16| 10527| 56|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
only showing top 2 rows

In [0]:
for x in [i for i in df1.columns][2:]:
  df1 = df1.withColumn(x,df1[x].cast(IntegerType()))

In [0]:
df1.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: integer (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [0]:
train,test = df1.randomSplit([0.7,0.3])

In [0]:
SI = StringIndexer(inputCols=['Private'],outputCols=['Private1'])
OHE = OneHotEncoder(inputCols=['Private1'],outputCols=['Private2'])

VA1 = VectorAssembler(inputCols=['Apps','Accept','Enroll','Top10perc','Top25perc','F_Undergrad','P_Undergrad','Outstate','Room_Board','Books',
                                 'Personal','PhD','Terminal','S_F_Ratio','perc_alumni','Expend','Grad_Rate'],outputCol='features')

DCT = DecisionTreeClassifier(labelCol='Private1')
RFC = RandomForestClassifier(labelCol='Private1')
GRA = GBTClassifier(labelCol='Private1')

In [0]:
pipe_dct = Pipeline(stages = [SI,VA1,DCT])
pipe_rfc = Pipeline(stages = [SI,VA1,RFC])
pipe_gra = Pipeline(stages = [SI,VA1,GRA])

In [0]:
train_data = pipe_dct.fit(train)
pred = train_data.transform(test)
Eval_data = pred.select(['Private1','prediction'])

In [0]:
train_data1 = pipe_rfc.fit(train)
pred1 = train_data1.transform(test)
Eval_data1 = pred1.select(['Private1','prediction'])

In [0]:
train_data2 = pipe_gra.fit(train)
pred2 = train_data2.transform(test)
Eval_data2 = pred2.select(['Private1','prediction'])

In [0]:
BCE = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='Private1',metricName='accuracy')
BCE1 = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='Private1',metricName='accuracy')
BCE2 = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='Private1',metricName='accuracy')


In [0]:
BCE.evaluate(Eval_data)

Out[171]: 0.8995983935742972

In [0]:
BCE1.evaluate(Eval_data1)

Out[172]: 0.9477911646586346

In [0]:
BCE.evaluate(Eval_data2)

Out[173]: 0.9036144578313253

In [0]:
pred2.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: integer (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)
-- Private1: double (nullable = false)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [0]:
pred.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: integer (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)
-- Private1: double (nullable = false)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [0]:
SF.translate()

Out[17]: <module 'pyspark.sql.functions' from '/databricks/spark/python/pyspark/sql/functions.py'>

In [0]:
SSS = SparkSession.builder.appName('RFC').getOrCreate()

In [0]:
df=SSS.read.csv('/FileStore/tables/dog_food.csv',inferSchema=True,header=True)

In [0]:
df.show()

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
 10| 3|13.0| 9| 1.0|
 8| 5|14.0| 5| 1.0|
 5| 8|12.0| 8| 1.0|
 6| 5|12.0| 9| 1.0|
 3| 3|12.0| 1| 1.0|
 9| 8|11.0| 3| 1.0|
 1| 10|12.0| 3| 1.0|
 1| 5|13.0| 10| 1.0|
 2| 10|12.0| 6| 1.0|
 1| 10|11.0| 4| 1.0|
 5| 3|12.0| 2| 1.0|
 4| 9|11.0| 8| 1.0|
 5| 1|11.0| 1| 1.0|
 4| 9|12.0| 10| 1.0|
 5| 8|10.0| 9| 1.0|
+---+---+----+---+-------+
only showing top 20 rows

In [0]:
VA = VectorAssembler(inputCols=['A','B','C','D'],outputCol='features')

In [0]:
df = VA.transform(df)
data  = df.select(['features','Spoiled'])

In [0]:
data.show()

+-------------------+-------+
 features|Spoiled|
+-------------------+-------+
 [4.0,2.0,12.0,3.0]| 1.0|
 [5.0,6.0,12.0,7.0]| 1.0|
 [6.0,2.0,13.0,6.0]| 1.0|
 [4.0,2.0,12.0,1.0]| 1.0|
 [4.0,2.0,12.0,3.0]| 1.0|
[10.0,3.0,13.0,9.0]| 1.0|
 [8.0,5.0,14.0,5.0]| 1.0|
 [5.0,8.0,12.0,8.0]| 1.0|
 [6.0,5.0,12.0,9.0]| 1.0|
 [3.0,3.0,12.0,1.0]| 1.0|
 [9.0,8.0,11.0,3.0]| 1.0|
[1.0,10.0,12.0,3.0]| 1.0|
[1.0,5.0,13.0,10.0]| 1.0|
[2.0,10.0,12.0,6.0]| 1.0|
[1.0,10.0,11.0,4.0]| 1.0|
 [5.0,3.0,12.0,2.0]| 1.0|
 [4.0,9.0,11.0,8.0]| 1.0|
 [5.0,1.0,11.0,1.0]| 1.0|
[4.0,9.0,12.0,10.0]| 1.0|
 [5.0,8.0,10.0,9.0]| 1.0|
+-------------------+-------+
only showing top 20 rows

In [0]:
RFC = RandomForestClassifier(featuresCol='features',labelCol='Spoiled')

In [0]:
trainmodel = RFC.fit(data)

In [0]:
trainmodel.featureImportances

Out[50]: SparseVector(4, {0: 0.0189, 1: 0.0141, 2: 0.9384, 3: 0.0287})